In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
!unrar x /content/gdrive/MyDrive/denemekaps3.rar

In [ ]:
import numpy as np
from keras import layers, models, optimizers
from keras import backend as K
from keras.utils import to_categorical
from keras.layers import *
from keras.utils import plot_model
import matplotlib.pyplot as plt
import cv2
import os
import splitfolders
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from skimage.color import rgb2gray
def load_mri():
    yol = '/content/denemekaps/'
    path = yol
    data = []
    label = []
    Files = ['herniyok', 'resim2']
    label_val = 0

    for files in Files:
        cpath = os.path.join(path, files)
        cpath = os.path.join(cpath)

        for img in sorted(os.listdir(cpath)):

          image_array = cv2.imread(os.path.join(cpath, img), cv2.IMREAD_COLOR)
          image_array=cv2.resize(image_array,(320,320))
          image_array=np.uint8(image_array)
          data.append(image_array)
          label.append(label_val)
        label_val = 1

    data=np.asarray(data)
    label =np.asarray(label,dtype=np.uint8)

    x_train, x_test, y_train, y_test = train_test_split(data, label, test_size=0.3, random_state=42)

    x_train = x_train.reshape(-1, 320, 320,3 ).astype('float32') / 255.
    x_test = x_test.reshape(-1, 320, 320,3 ).astype('float32') / 255.
    y_train = to_categorical(y_train.astype('float32'))
    y_test = to_categorical(y_test.astype('float32'))

    return (x_train, y_train), (x_test, y_test)

In [ ]:
def load_data():
    yol = '/content/denemekaps/'
    path = yol
    data = []
    label = []
    Files = ['herniyok', 'resim2']
    label_val = 0

    for files in Files:
        cpath = os.path.join(path, files)
        cpath = os.path.join(cpath)

        for img in sorted(os.listdir(cpath)):

          image_array = cv2.imread(os.path.join(cpath, img), cv2.IMREAD_COLOR)
          image_array=cv2.resize(image_array,(320,320))
          image_array=np.uint8(image_array)
          data.append(image_array)
          label.append(label_val)
        label_val = 1

    data=np.asarray(data)
    label =np.asarray(label,dtype=np.uint8)

    data = data.reshape(-1, 320, 320,3 ).astype('float32') / 255.
    label = to_categorical(label.astype('float32'))

    return data, label

In [ ]:
import numpy as np
import pdb
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from scipy.optimize import minimize

def gwo_optimize(X_train, y_train, X_test, y_test):
    search_agents = 5
    max_iter = 50
 #   pdb.set_trace()
    # GWO Başlangıç Pozisyonları
    positions = np.random.uniform(low=-1, high=1, size=(search_agents, X_train.shape[1]))

    # GWO Ana Döngü
    for iteration in range(max_iter):
        # GWO Popülasyonunu Değerlendir
        fitness = evaluate_fitness(positions, X_train, y_train, X_test, y_test)

        # GWO Algoritması
        alpha, beta, delta = get_alpha_beta_delta(positions, fitness)
        a = 2 - 2 * iteration / max_iter  # Azalma katsayısı

        # Güncelleme formülü
        positions = (alpha + beta + delta) / 3 + a * (np.random.uniform(size=positions.shape) - 0.5)

    # GWO Optimizasyon Sonuçları
    best_position = positions[np.argmin(fitness)]
    return best_position

# GWO Fitness Fonksiyonu
def evaluate_fitness(positions, X_train, y_train, X_test, y_test):
    fitness = []
    for pos in positions:
        # Her bir pozisyon için GWO ile optimize edilen öznitelikleri kullan
        optimized_features = X_train * pos

        # MLP Sınıflandırıcı ile eğitim
        clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=50)
        try:
            clf.fit(optimized_features, y_train.argmax(axis=1))
        except Exception as e:
            print(f"Hata: {e}")
            return np.zeros(len(y_test))  # Hataya düşerse fitness değerini sıfır olarak döndür

        # Test verileri üzerinde tahmin yap
        predictions = clf.predict(X_test * pos)

        acc = accuracy_score(y_test.argmax(axis=1), predictions)
        fitness.append(acc)

    return np.array(fitness)

# GWO Algoritması için Alpha, Beta, Delta'nın belirlenmesi
def get_alpha_beta_delta(positions, fitness):
    sorted_indices = np.argsort(fitness)
    alpha, beta, delta = positions[sorted_indices[:3]]
    return alpha, beta, delta

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.applications import ResNet50

input_shape = (320, 320, 3)

input_layer = Input(shape=input_shape)

resnet_model = ResNet50(include_top=False, input_shape=input_shape, weights='imagenet')

x = resnet_model(input_layer)

x = Flatten()(x)

x = Dense(1024, activation='relu')(x)

output_layer = Dense(2, activation='softmax')(x)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense,Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from scipy.optimize import minimize
from tensorflow.keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from scipy.optimize import minimize
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint

weight_path = "{}_resnetortak.weights.h5".format('BEST')

epochs = 50
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                   patience=4,
                                   verbose=1, mode='min', lr=0.001, cooldown=2, min_lr=0.0001)
checkpoint = ModelCheckpoint(
    weight_path,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='auto',
    save_weights_only=True  # .h5 formatı için bu parametre gereklidir
)

callbacks_list = [checkpoint]
history = model.fit(
  X_train.astype(np.float32), y_train.astype(np.float32),
  batch_size=4,
  shuffle=False,
  validation_data=(x_val.astype(np.float32), y_val.astype(np.float32)),
  epochs=epochs,
  callbacks=callbacks_list
)


In [ ]:
from tensorflow.keras.models import load_model
fc_output_model = Model(inputs=model.input, outputs=model.layers[-2].output)
model.save(weight_path)
best_model = load_model(weight_path)

input_example = X_train[0:1].astype(np.float32)

best_model.layers[-2].output.shape
fully_connected_outputs = model.layers[-2].output
data,label=load_data()
X_train_fc_output = fc_output_model.predict(X_train)
data_fc_output = fc_output_model.predict(data)
X_val_fc_output = fc_output_model.predict(x_val)
X_test_fc_output = fc_output_model.predict(X_test)
print("fully_connected_outputs etiketleri boyutu:", fully_connected_outputs.shape)
print("X_train_fc_output etiketleri boyutu:", X_train_fc_output.shape)
print("X_val_fc_output etiketleri boyutu:", X_val_fc_output.shape)
print("X_test_fc_output etiketleri boyutu:", X_test_fc_output.shape)
print("data_fc_output etiketleri boyutu:", data_fc_output.shape)

In [ ]:
import matplotlib.pyplot as plt

# Eğitim ve doğrulama kayıpları
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Eğitim ve doğrulama doğrulukları
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Epoch sayısı
epochs = range(1, len(train_loss) + 1)

# Kayıp Eğrisi
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, 'b', label='Eğitim Kaybı')
plt.plot(epochs, val_loss, 'r', label='Doğrulama Kaybı')
plt.title('Eğitim ve Doğrulama Kayıpları')
plt.xlabel('Epochs')
plt.ylabel('Kayıp')
plt.legend()

# Doğruluk Eğrisi
plt.subplot(1, 2, 2)
plt.plot(epochs, train_acc, 'b', label='Eğitim Doğruluğu')
plt.plot(epochs, val_acc, 'r', label='Doğrulama Doğruluğu')
plt.title('Eğitim ve Doğrulama Doğrulukları')
plt.xlabel('Epochs')
plt.ylabel('Doğruluk')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Optimizasyon İşlemi
best_deger = gwo_optimize(X_train_fc_output, y_train, X_val_fc_output, y_val)
print("Optimize Edilen:", best_deger.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

fold_accuracies = []
train_accuracies = []
loss_curves = []
val_loss_curves = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(data_fc_output)):
    data_fold, X_val_fold = data_fc_output[train_index], data_fc_output[val_index]
    y_train_fold, y_val_fold = label[train_index], label[val_index]

    mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=50, warm_start=True)

    mlp_model.fit(data_fold * best_deger, y_train_fold.argmax(axis=1))

    train_accuracy = mlp_model.score(data_fold * best_deger, y_train_fold.argmax(axis=1))
    train_accuracies.append(train_accuracy)

    val_predictions = mlp_model.predict(X_val_fold * best_deger)
    val_accuracy = accuracy_score(y_val_fold.argmax(axis=1), val_predictions)
    fold_accuracies.append(val_accuracy)

    loss_curves.append(mlp_model.loss_curve_)

    print(f"Fold Accuracy: {val_accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_val_fold.argmax(axis=1), val_predictions, target_names=['noherni', 'herni']))

    accuracy = accuracy_score(y_val_fold.argmax(axis=1), val_predictions)
    precision = precision_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')
    recall = recall_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')
    f1 = f1_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    cm = confusion_matrix(y_val_fold.argmax(axis=1), val_predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['noherni', 'herni'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Fold Confusion Matrix {fold+1}")
    plt.show()


print(f"Average Validation Accuracy: {np.mean(fold_accuracies):.4f}")
print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average Precision: {np.mean(precisions):.4f}")
print(f"Average Recall: {np.mean(recalls):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

fold_accuracies = []
train_accuracies = []
loss_curves = []
val_loss_curves = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(data_fc_output)):

    data_fold, X_val_fold = data_fc_output[train_index], data_fc_output[val_index]
    y_train_fold, y_val_fold = label[train_index], label[val_index]

    mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=50, warm_start=True,activation='identity')

    mlp_model.fit(data_fold * best_deger, y_train_fold.argmax(axis=1))

    train_accuracy = mlp_model.score(data_fold * best_deger, y_train_fold.argmax(axis=1))
    train_accuracies.append(train_accuracy)

    val_predictions = mlp_model.predict(X_val_fold * best_deger)
    val_accuracy = accuracy_score(y_val_fold.argmax(axis=1), val_predictions)
    fold_accuracies.append(val_accuracy)

    loss_curves.append(mlp_model.loss_curve_)

    print(f"Fold Accuracy: {val_accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_val_fold.argmax(axis=1), val_predictions, target_names=['noherni', 'herni']))
    accuracy = accuracy_score(y_val_fold.argmax(axis=1), val_predictions)
    precision = precision_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')
    recall = recall_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')
    f1 = f1_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    cm = confusion_matrix(y_val_fold.argmax(axis=1), val_predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['noherni', 'herni'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Fold Confusion Matrix {fold+1}")
    plt.show()


print(f"Average Validation Accuracy: {np.mean(fold_accuracies):.4f}")

print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average Precision: {np.mean(precisions):.4f}")
print(f"Average Recall: {np.mean(recalls):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

fold_accuracies = []
train_accuracies = []
loss_curves = []
val_loss_curves = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(data_fc_output)):

    data_fold, X_val_fold = data_fc_output[train_index], data_fc_output[val_index]
    y_train_fold, y_val_fold = label[train_index], label[val_index]

    mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=50, warm_start=True,activation='tanh')

    mlp_model.fit(data_fold * best_deger, y_train_fold.argmax(axis=1))

    train_accuracy = mlp_model.score(data_fold * best_deger, y_train_fold.argmax(axis=1))
    train_accuracies.append(train_accuracy)

    val_predictions = mlp_model.predict(X_val_fold * best_deger)
    val_accuracy = accuracy_score(y_val_fold.argmax(axis=1), val_predictions)
    fold_accuracies.append(val_accuracy)

    loss_curves.append(mlp_model.loss_curve_)

    print(f"Fold Accuracy: {val_accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_val_fold.argmax(axis=1), val_predictions, target_names=['noherni', 'herni']))
    accuracy = accuracy_score(y_val_fold.argmax(axis=1), val_predictions)
    precision = precision_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')
    recall = recall_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')
    f1 = f1_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)
    cm = confusion_matrix(y_val_fold.argmax(axis=1), val_predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['noherni', 'herni'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Fold Confusion Matrix {fold+1}")
    plt.show()


print(f"Average Validation Accuracy: {np.mean(fold_accuracies):.4f}")
print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average Precision: {np.mean(precisions):.4f}")
print(f"Average Recall: {np.mean(recalls):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold cross-validation

fold_accuracies = []
train_accuracies = []
loss_curves = []
val_loss_curves = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(data_fc_output)):

    data_fold, X_val_fold = data_fc_output[train_index], data_fc_output[val_index]
    y_train_fold, y_val_fold = label[train_index], label[val_index]

    mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=50, warm_start=True,activation='logistic')

    mlp_model.fit(data_fold * best_deger, y_train_fold.argmax(axis=1))

    train_accuracy = mlp_model.score(data_fold * best_deger, y_train_fold.argmax(axis=1))
    train_accuracies.append(train_accuracy)

    val_predictions = mlp_model.predict(X_val_fold * best_deger)
    val_accuracy = accuracy_score(y_val_fold.argmax(axis=1), val_predictions)
    fold_accuracies.append(val_accuracy)

    loss_curves.append(mlp_model.loss_curve_)

    print(f"Fold Accuracy: {val_accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_val_fold.argmax(axis=1), val_predictions, target_names=['noherni', 'herni']))

    accuracy = accuracy_score(y_val_fold.argmax(axis=1), val_predictions)
    precision = precision_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')
    recall = recall_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')
    f1 = f1_score(y_val_fold.argmax(axis=1), val_predictions, average='weighted')

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    cm = confusion_matrix(y_val_fold.argmax(axis=1), val_predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['noherni', 'herni'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Fold Confusion Matrix {fold+1}")
    plt.show()


print(f"Average Validation Accuracy: {np.mean(fold_accuracies):.4f}")
print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average Precision: {np.mean(precisions):.4f}")
print(f"Average Recall: {np.mean(recalls):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")


In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

kf = KFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []
precisions = []
recalls = []
f1_scores = []

for fold, (train_index, val_index) in enumerate(kf.split(data_fc_output)):
    print(f"Fold {fold + 1}...")

    data_fold, X_val_fold = data_fc_output[train_index], data_fc_output[val_index]
    label_fold, y_val_fold = label[train_index], label[val_index]

    svm_model = SVC(kernel='poly', C=1)

    svm_model.fit(data_fold * best_deger, label_fold.argmax(axis=1))

    train_accuracy = svm_model.score(data_fold * best_deger, label_fold.argmax(axis=1))
    print(f"Train Accuracy: {train_accuracy:.4f}")

    svm_predictions = svm_model.predict(X_val_fold * best_deger)

    accuracy = accuracy_score(y_val_fold.argmax(axis=1), svm_predictions)
    precision = precision_score(y_val_fold.argmax(axis=1), svm_predictions, average='weighted')
    recall = recall_score(y_val_fold.argmax(axis=1), svm_predictions, average='weighted')
    f1 = f1_score(y_val_fold.argmax(axis=1), svm_predictions, average='weighted')

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1_scores.append(f1)

    cm = confusion_matrix(y_val_fold.argmax(axis=1), svm_predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['noherni', 'herni'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix - Fold {fold + 1}")
    plt.show()

print("\nCross-Validation Results:")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average Precision: {np.mean(precisions):.4f}")
print(f"Average Recall: {np.mean(recalls):.4f}")
print(f"Average F1 Score: {np.mean(f1_scores):.4f}")
